In [ ]:
import pyglet, random, math, os, sys, numpy as np, copy
from pyglet.window import key, FPSDisplay
from pyglet import image, shapes

# begin coordinat left down corner
window = pyglet.window.Window(1200,400,"One oscillator",resizable = False)
batch = pyglet.graphics.Batch()
fps_display = FPSDisplay(window)
fps_display.label = pyglet.text.Label(font_name='Arial', font_size=12, x=20, y=20, color = (0,0,0,200))
label = pyglet.text.Label('A.Nurutdinov (c)', font_name='Arial', font_size=12, x=100, y=20, color = (0,0,0,200))
background = pyglet.sprite.Sprite(pyglet.image.load('fig3.png'), x = 0, y = 0, batch=batch)
pyglet.gl.glClearColor(1,1,1,1)

# ===================== Parallel fabers =========================== 
N = 260               # time scan length

PFy  = 0.; PF2y  = 0.             # y coordinates of each of the bodies
PFvy = 0.; PF2vy = 0.             # y speed of each body (first derivative of)
PFay = 0.; PF2ay = 0.             # y acceleration of each body (second derivative of)

PFcm = 0.0; PF2cm = 0.05          # attenuation rate (braking signal)
PFkm = 0.4; PF2km = 0.25          # degree of elasticity (generation frequency)

PFmem  = np.full(N,.0) # memory state with depth N
PF2mem = np.full(N,.0) # memory state with depth N

ON = False  # running the model 
# ======================= prepare procedure =============================
BodyC = [shapes.Rectangle(0,0,3,3,color=(200, 100, 100), batch=batch) for i in range(N)]  # coordinates
BodyC[0].width = 20; BodyC[0].height = 20; BodyC[0].opacity = 200
BodyC[0].color = (200, 100, 100); BodyC[0].anchor_position = (10,10)
    
Body2C = [shapes.Rectangle(0,0,3,3,color=(100, 100, 200), batch=batch) for i in range(N)]  # coordinates
Body2C[0].width = 20; Body2C[0].height = 20; Body2C[0].opacity = 200
Body2C[0].color = (100, 100, 200); Body2C[0].anchor_position = (10,10)

for i in range(N):
    BodyC[i].x = 45 + 2*i
    BodyC[i].y = PFmem[i]    
    
    Body2C[i].x = 630 + 2*i
    Body2C[i].y = PFmem[i]
# ======================= drawing procedure =============================
@window.event
def on_draw():
    window.clear()
    fps_display.draw()
    batch.draw()
    #label.draw()
# ======================= mouse press ============================
@window.event
def on_mouse_press(x, y, button, modifiers):
    global ON
    if button == pyglet.window.mouse.LEFT:
        ON = True
# ======================= update procedure ============================
def update(dt):  
    global BodyС, PFmem, PFay, PFvy, PFy, Body2С, PF2mem, PF2ay, PF2vy, PF2y, ON
    
    for i in range(N):        
        BodyC[i].y  = 200+PFmem[i]
        Body2C[i].y = 200+PF2mem[i]
    
    if ON:                                                             # excitation
        PFvy =  85.
        PF2vy = 85.
        ON = False

    TC = .002                                                       # time correlation
    for j in range(100):
        PFay = -PFcm*PFvy -PFkm*PFy
        PFvy += PFay*TC
        PFy  += PFvy*TC
        
        PF2ay = -PF2cm*PF2vy -PF2km*PF2y
        PF2vy += PF2ay*TC
        PF2y  += PF2vy*TC
    
    PFmem = np.roll(PFmem, 1)
    PFmem[0] = PFy
    
    PF2mem = np.roll(PF2mem, 1)
    PF2mem[0] = PF2y
    
    #label.text = str(max(PFy))     
# =========================== MAIN LOOP ============================
if __name__ == "__main__":
    pyglet.clock.schedule_interval(update, 1/50)
    pyglet.app.run()
    pyglet.clock.unschedule(update)   # necessary to restart the countdown of time intervals
#========================================================================